# Structured Output Parser

`StructuredOutputParser` là một công cụ giá trị để định dạng phản hồi của Mô hình Ngôn ngữ Lớn (LLM) thành cấu trúc từ điển, cho phép trả về nhiều trường dưới dạng cặp key/value.
Trong khi Pydantic và JSON parsers cung cấp các khả năng mạnh mẽ, `StructuredOutputParser` đặc biệt hiệu quả đối với các mô hình ít mạnh mẽ hơn, chẳng hạn như các mô hình cục bộ với ít tham số hơn. Nó đặc biệt có lợi cho các mô hình có trí tuệ thấp hơn so với các mô hình tiên tiến như GPT hoặc Claude.
Bằng cách sử dụng `StructuredOutputParser`, các nhà phát triển có thể duy trì tính toàn vẹn và nhất quán của dữ liệu trên nhiều ứng dụng LLM khác nhau, ngay cả khi hoạt động với các mô hình có số lượng tham số giảm.


In [1]:
# Setup environment
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv(override=True, dotenv_path="../.env")

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

## Implementing Structured Output Parser

### Using ResponseSchema with StructuredOutputParser
*   Define a response schema using the `ResponseSchema` class to include the answer to the user's question and a `description` of the source (website) used.

*   Initialize `StructuredOutputParser` with `response_schemas` to structure the output according to the defined response schema.

**[Note]**
When using local models, Pydantic parsers may frequently fail to work properly. In such cases, using `StructuredOutputParser` can be a good alternative solution.

In [6]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

# Response to the user's question
response_schemas = [
    ResponseSchema(name="answer", description="Answer to the user's question"),
    ResponseSchema(
        name="source",
        description="The `source` used to answer the user's question, which should be a `website URL`.",
    ),
]
# Initialize the structured output parser based on the response schemas
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

### Embedding Response Schemas into Prompts 

Create a `PromptTemplate` to format user questions and embed parsing instructions for structured outputs.

In [7]:
from langchain_core.prompts import PromptTemplate
# Parse the format instructions.
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    # Set up the template to answer the user's question as best as possible.
    template="answer the user's question as best as possible.\n{format_instructions}\n{question}",
    # Use 'question' as the input variable.
    input_variables=["question"],
    # Use 'format_instructions' as a partial variable.
    partial_variables={"format_instructions": format_instructions},
)

In [8]:
chain = prompt | llm | output_parser
chain.invoke({"question": "What is the largest desert in the world?"})

{'answer': 'The largest desert in the world is the Antarctic Desert.',
 'source': 'https://www.britannica.com/place/desert'}

### Using Streamed Outputs

Use the `chain.stream` method to receive a streaming response to the `question` , "How many players are on a soccer team?"

In [9]:
for s in chain.stream({"question": "How many players are on a soccer team?"}):
    # Stream the output
    print(s)

{'answer': 'A standard soccer team consists of 11 players on the field at a time.', 'source': 'https://www.fifa.com/who-we-are/news/what-are-the-rules-of-soccer'}
